# Como fazer chamadas simples com API?
Primeiro escolhemos o modelo (ex.: ChatGPT ou Gemini). Caso escolha ChatGPT, execute *pip install openai*, caso escolha o Gemini, execute *pip install google*.

In [1]:
# LLMs
from openai import OpenAI
from google import genai
from google.genai import types

# Para validação de campos e geração de uma resposta estruturada
from pydantic import BaseModel, Field
from enum import Enum
import json

# Para a leitura das chaves no arquivo ".env"
from dotenv import load_dotenv, dotenv_values

# Biblioteca utilizada para a leitura de dataframes
import pandas as pd

In [3]:
# Protegendo as chaves de API e carregando-as em variáveis
config = dotenv_values("../.env")
openai_api_key = config["OPENAI_API_KEY"]
gemini_api_key = config["GEMINI_API_KEY"]

In [4]:
# Alimentando o client com as devidas chaves de api
client_openai = OpenAI(api_key=openai_api_key)
client_gemini = genai.Client(api_key=gemini_api_key)

In [6]:
# Ambos as funções abaixo possuem o mesmo funcionamento: 
# 1 - Você passa um contexto/prompt (context)
# 2 - Você passa uma pergunta ou qulquer coisa que queira saber (input)
# As funções fazem uma chamada simples para os modelos de LLM e a resposta obtida é retornada

# Função que utiliza a API da OpenAI
def response_openai(context:str, input:str) -> str:
    response = client_openai.responses.create(
        model="gpt-4.1-mini",
        instructions=context,
        input= input,
        temperature=0.5
    )
    return response.output_text

# Função que utiliza a API do Gemini
def response_gemini(context:str, input:str) -> str:
        
    response = client_gemini.models.generate_content(
        model="gemini-2.5-flash", 
        config=types.GenerateContentConfig(
            system_instruction=context,
            temperature=0.5),
        contents=input
    )
    return response.text

In [7]:
contexto = """
Capitu e Bentinho são os personagens centrais de Dom Casmurro, 
de Machado de Assis, um romance realista que explora o ciúme doentio de Bentinho 
e a dúvida sobre a suposta traição de Capitu com seu melhor amigo, Escobar, 
num contexto social do Rio de Janeiro do século XIX, marcado por aparências, 
classes sociais e papéis femininos restritos, onde a narrativa enviesada de Bentinho, 
já um homem amargurado, impede o leitor de saber a verdade absoluta, 
deixando a ambiguidade da inocência ou culpa de Capitu como o grande enigma da obra. 
"""

input = "Capitu traiu Bentinho?"

### Teste OpenAI

In [8]:
response_openai(contexto, input)

'A questão sobre se Capitu traiu Bentinho é um dos grandes mistérios da literatura brasileira e um dos temas centrais de *Dom Casmurro*, de Machado de Assis. O romance é narrado por Bentinho, que, já adulto e amargurado, relata sua história e suas suspeitas de traição.\n\nNo entanto, o próprio Bentinho nunca apresenta uma prova concreta da infidelidade de Capitu. A narrativa é marcada pela subjetividade e pelo ciúme doentio do narrador, o que torna difícil ao leitor confiar plenamente em sua versão dos fatos. Machado de Assis constrói a história de forma a deixar essa dúvida em aberto, explorando a ambiguidade e a complexidade das relações humanas.\n\nPortanto, não há uma resposta definitiva no texto. A dúvida sobre a fidelidade de Capitu permanece como um enigma, e cada leitor pode interpretar a história de acordo com sua visão. Essa ambiguidade é justamente o que torna *Dom Casmurro* uma obra tão rica e discutida.'

### Teste Gemini

In [9]:
response_gemini(contexto, input)

'Essa é a grande questão e o maior enigma de "Dom Casmurro"!\n\n**Não há uma resposta definitiva na obra.** Machado de Assis construiu a narrativa de forma que a verdade sobre a suposta traição de Capitu permaneça ambígua.\n\nAqui estão os pontos-chave:\n\n1.  **Narrador Não Confiável:** A história é contada exclusivamente por Bentinho, já um homem velho e amargurado, consumido pelo ciúme. Sua perspectiva é enviesada e ele interpreta todos os acontecimentos e as ações de Capitu através da lente de sua desconfiança.\n2.  **Falta de Provas Concretas:** Bentinho nunca apresenta provas irrefutáveis da traição. Suas "evidências" são sempre subjetivas: o olhar de Capitu ("olhos de ressaca"), a semelhança (ou a percepção de semelhança) de seu filho Ezequiel com Escobar, e suas próprias deduções baseadas em ciúme.\n3.  **A Ambiguidade é a Essência:** O objetivo de Machado de Assis não era dar uma resposta, mas sim fazer o leitor duvidar, refletir sobre a natureza do ciúme, da verdade e da perc

## Como obtemos uma resposta estruturada na forma de JSON?
Vamos supor que você quer analisar um processo judicial envolvendo tráfico de drogas de forma rápida e eficiente. Para isso você precisa extrair informações como: 

* O processo se trata de tráfico de drogas?
* Qual o tipo de droga envolvida?
* Qual a quantidade do tipo de droga envolvida?
* Qual o sexo da pessoa envolvida?
* Qual o tipo de pena?
* Qual o tempo da pena em meses? 

Para essa tarefa precisaremos utilizar pydantic para garantir uma validação dos campos que devem ser retornados pelo LLM e precisamos mudar um pouquinho a estrutura das nossas funções de chamadas.

In [10]:
# Set da resposta esperado pelo LLM

class Escopo(Enum): # Enum garante que saia apenas uma das opções presentes no escopo da classe
    SIM = "sim"
    NAO = "não"

class Maconha(Enum):
    SIM = "sim"
    NAO = "não"

class Cocaina(Enum):
    SIM = "sim"
    NAO = "nao"

class Crack(Enum):
    SIM = "sim"
    NAO = "nao"

class Sexo(Enum):
    MASC = "masculino"
    FEM = "feminino"
    NULL = "não informado"

class Reincidente(Enum):
    SIM = "sim"
    NAO = "não"
    NULL = "não informado"

class Decisao(Enum):
    PROCEDENTE = "procedente"
    IMPROCEDENTE = "improcedente"
    PARCIAL = "parcialmente procedente"

class Pena(Enum):
    FECHADO = "fechado"
    ABERTO = "aberto"
    SEMIABERTO = "semiaberto"


class Drogas(BaseModel): # BaseModel garante a validação dos campos referentes aos atributos da classe 
    escopo: Escopo
    maconha: Maconha
    cocaina: Cocaina
    crack: Crack
    qtd_maconha: float
    qtd_cocaina: float
    qtd_crack: float
    sexo: Sexo
    reincidente: Reincidente
    decisao: Decisao
    tipo_pena: Pena
    tempo: float

In [11]:
def classificador_openai(context:str, input:str, estrutura:Drogas) -> dict:
    response = client_openai.responses.parse(  # mudar aqui de .create() para .parse()
        model="gpt-4.1-mini",
        instructions=context,
        input=input,
        temperature=0.0,
        text_format = estrutura # mudar aqui para o schema de classe
    )
    # mudar aqui de "response.output_text" para "response.output_parsed"
    return response.output_parsed.model_dump(mode="json")


def classificador_gemini(prompt:str, input:str, estrutura:Drogas) -> dict:
    content = f"{prompt}\n\nProcesso:\n{input}"
    response = client_gemini.models.generate_content( 
        model="gemini-2.5-flash",
        contents=content,
        config={
        "response_mime_type": "application/json",
        "response_json_schema": estrutura.model_json_schema(),
    },  
    )

    
    # mudar aqui de "response.text" por:
    result = response.candidates[0].content.parts[0].text
    result = json.loads(result)
    return result

#### Qual a diferença entre ```.parse()``` e ```.create()``` ?

* ``.create()`` gera uma resposta bruta do modelo
* ``.parse()`` gera resposta estruturada 

In [12]:
# É necessário também um bom prompt para garantir a saída de uma resposta que segue a estrutura que nós buscamos
prompt = """
Você é um assistente de inteligência artificial que auxilia na anotação de sentenças judiciais do Tribunal de Justiça de São Paulo em processos envolvendo tráfico de drogas.

Você receberá o texto da sentença e deverá retornar um arquivo JSON, seguindo as regras abaixo:

- O processo faz parte do escopo da pesquisa? O caso deve: Ser um caso relacionado a tráfico de drogas; Envolver porte de maconha, crack ou cocaína; Envolver apenas uma pessoa acusada.
- Quantidade de maconha em gramas: Preencha apenas os números. Coloque 0 se o caso não envolve essa droga. Use "," como separador decimal. Se a decisão não menciona a quantidade em gramas, faça a conversão, seguindo a regra: 1 porção = 2 gramas.
- Quantidade de cocaína em gramas: Preencha apenas os números. Coloque 0 se o caso não envolve essa droga. Use "," como separador decimal. Se a decisão não menciona a quantidade em gramas, faça a conversão, seguindo a regra: 1 porção ou pino = 0,5 grama.
- Quantidade de crack em gramas: Preencha apenas os números. Coloque 0 se o caso não envolve essa droga. Use "," como separador decimal. Se a decisão não menciona a quantidade em gramas, faça a conversão, seguindo a regra: 1 porção = 0,1 grama.
- Decisão: pode ser procedente (condenação), improcedente / punibilidade extinta, ou parcialmente procedente / advertência.
- Tipo de pena: pode ser fechado, semiaberto ou aberto.
- Tempo da pena (em meses): Preencha apenas os números. Converta o tempo em meses. Por exemplo, 2 anos e 7 meses = 31 meses.

Retorne um arquivo JSON com as seguintes informações:

{
 "escopo": "sim/não",
 "maconha": "sim/não",
 "cocaina": "sim/não",
 "crack": "sim/não",
 "qtd_maconha": 0,
 "qtd_cocaina": 0,
 "qtd_crack": 0,
 "sexo": "masculino/feminino/não informado",
 "reincidente": "sim/não/não informado",
 "decisao": "procedente/improcedente/parcialmente procedente",
 "tipo_pena": "fechado/semiaberto/aberto",
 "tempo": 0
}
"""

## Vamos testar esses modelos na análise de alguns processos

In [13]:
drogas = pd.read_csv("https://github.com/jtrecenti/main-cdad2/releases/download/data/drogas.csv")
drogas

,file,processo,pagina,hora_coleta,duplicado,classe,assunto,magistrado,comarca,foro,vara,disponibilizacao,julgado,cd_doc
0,praticas/drogas/2024_08_19_20_56_38_224078_pag...,15020146520218260544,100,NaN,False,Ação Penal - Procedimento Ordinário,Tráfico de Drogas e Condutas Afins,ÉNDERSON DANILO SANTOS DE VASCONCELOS,Franco da Rocha,Foro de Franco da Rocha,Vara Criminal,2024-08-08,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO COM...,F40005C7H0000-198-PG5JUN-98805557
1,praticas/drogas/2024_08_19_20_56_38_224078_pag...,15028371020238260628,100,NaN,False,Ação Penal - Procedimento Ordinário,Tráfico de Drogas e Condutas Afins,Leticia Antunes Tavares,Itapecerica da Serra,Foro de Itapecerica da Serra,2ª Vara,2024-08-08,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO COM...,HG00016HR0000-268-PG5GRU-106766014
2,praticas/drogas/2024_08_19_20_56_38_224078_pag...,15011249120238260630,100,NaN,True,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,ELIZABETH SHALDERS DE OLIVEIRA ROXO,Santa Bárbara d'Oeste,Foro de Santa Bárbara d'Oeste,2ª Vara Criminal,2024-08-08,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO COM...,HI0001HTY0000-533-PG5CAMP-125799939
3,praticas/drogas/2024_08_19_20_56_38_224078_pag...,15004789120248260583,100,NaN,False,Auto de Prisão em Flagrante,Tráfico de Drogas e Condutas Afins,Antonio Roberto Sylla,Presidente Prudente,Foro de Presidente Prudente,1ª Vara Criminal,2024-08-08,etendida substituição da privativa de liberdad...,G70000NVM0000-482-PG5PP-89742935
4,praticas/drogas/2024_08_19_20_56_38_224078_pag...,15010211620248260318,100,NaN,False,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,GUSTAVO DE CASTRO CAMPOS,Leme,Foro de Leme,Vara Criminal,2024-08-08,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO COM...,8U00079KH0000-318-PG5JUN-98824640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,praticas/drogas/2024_08_19_20_57_03_893601_pag...,15002971920248260542,99,NaN,False,Ação Penal - Procedimento Ordinário,Tráfico de Drogas e Condutas Afins,ULISSES AUGUSTO PASCOLATI JUNIOR,Osasco,Foro de Osasco,4ª Vara Criminal,2024-08-08,gas se destinavam à venda para consumo de terc...,F20001KKG0000-405-PG5GRU-106225576
987,praticas/drogas/2024_08_19_20_57_03_893601_pag...,15009824120248260537,99,NaN,False,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Maria Isabel Rebello Pinho Dias,Diadema,Foro de Diadema,1ª Vara Criminal,2024-08-08,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO COM...,EX0002UR60000-161-PG5SORC-110490684
988,praticas/drogas/2024_08_19_20_57_03_893601_pag...,15003859320248260530,99,NaN,False,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Guaracy Sibille Leite,Ribeirão Preto,Foro de Ribeirão Preto,1ª Vara Criminal,2024-08-08,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO COM...,EQ0003UHZ0000-506-PG5RP-100664927
989,praticas/drogas/2024_08_19_20_57_03_893601_pag...,15023636820218260544,99,NaN,False,Ação Penal - Procedimento Ordinário,Tráfico de Drogas e Condutas Afins,ÉNDERSON DANILO SANTOS DE VASCONCELOS,Franco da Rocha,Foro de Franco da Rocha,Vara Criminal,2024-08-08,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO COM...,F40005M5T0000-198-PG5JUN-98798886


### Teste OpenAI

In [14]:
processo = drogas[drogas["processo"] == drogas["processo"][777]]["julgado"].iloc[0]
resultado_openai = classificador_openai(prompt, processo, Drogas)
resultado_openai

{'escopo': 'não',
 'maconha': 'sim',
 'cocaina': 'sim',
 'crack': 'nao',
 'qtd_maconha': 49.0,
 'qtd_cocaina': 39.0,
 'qtd_crack': 0.0,
 'sexo': 'não informado',
 'reincidente': 'não informado',
 'decisao': 'procedente',
 'tipo_pena': 'fechado',
 'tempo': 81.0}

### Teste Gemini

In [15]:
resultado_gemini = classificador_gemini(prompt, processo, Drogas)
resultado_gemini

{'escopo': 'não',
 'maconha': 'sim',
 'cocaina': 'sim',
 'crack': 'nao',
 'qtd_maconha': 49.7,
 'qtd_cocaina': 39.11,
 'qtd_crack': 0,
 'sexo': 'não informado',
 'reincidente': 'não informado',
 'decisao': 'procedente',
 'tipo_pena': 'fechado',
 'tempo': 81}